In [1]:
%load_ext autoreload

In [2]:
import pandas as pd
from datetime import datetime
import pyarrow.parquet as pq # Used to read the data
import os 
import numpy as np
from keras.layers import * # Keras is the most friendly Neural Network library, this Kernel use a lot of layers classes
from keras.models import Model
from tqdm import tqdm # Processing time measurement
from sklearn.model_selection import train_test_split 
from keras import backend as K # The backend give us access to tensorflow operations and allow us to create the Attention class
from keras import optimizers # Allow us to access the Adam class to modify some parameters
from sklearn.model_selection import GridSearchCV, StratifiedKFold # Used to use Kfold to train our model
from keras.callbacks import * # This object helps the model to train in a smarter way, avoiding overfitting
from kit.attention import Attention
from sklearn.metrics import matthews_corrcoef
import matplotlib.pyplot as plt
from kit.utils import mcc_k
from numba import jit
from sklearn.metrics import roc_curve, roc_auc_score
import pickle
import time
from dask import dataframe as dd
from dask.multiprocessing import get
from dask import delayed
from dask import compute
from multiprocessing import cpu_count
from dask.distributed import Client
nCores = cpu_count()
import tensorflow as tf
from sklearn.model_selection import ParameterGrid
from keras.utils import multi_gpu_model

Using TensorFlow backend.


In [3]:
%autoreload
from kit.genesis import FeatureExtractor, data_prep
from kit.utils import gen_name, threshold_search, gen_name_v2

In [4]:
def check_hardware():
    list_of_gpus = K.tensorflow_backend._get_available_gpus()
    no_gpu = len([item for item in list_of_gpus if 'GPU' in item])
    if no_gpu > 0:
        print('Found {} GPUs.'.format(no_gpu))
    else:
        print('No GPU found!!')
    
    print('Found {} CPU cores.'.format(nCores))    

In [5]:
def load_data():
    meta_train = pd.read_csv('input/metadata_train.csv')
    meta_train = meta_train.set_index(['id_measurement', 'phase'])
    df_train= pq.read_pandas('input/train.parquet').to_pandas()
    return meta_train, df_train

In [6]:
def model_one(input_shape, kw):
    inp = Input(shape=(input_shape[1], input_shape[2],))
    x = Bidirectional(LSTM(kw['hu1'], return_sequences=True))(inp)
    x = Dropout(rate = kw['dr1'])(x)
    x = Bidirectional(LSTM(kw['hu2'], return_sequences=True))(x)    
    x = Attention(input_shape[1])(x)
    x = Dense(kw['de1'], activation="relu")(x)
    x = Dropout(rate =kw['dr2'])(x)
    x = Dense(1, activation="sigmoid")(x)
    if kw['parallel']:
        model = Model(inputs=inp, outputs=x)
        parallel_model = multi_gpu_model(model)
        parallel_model.compile(loss='binary_crossentropy', optimizer='adam', metrics=[mcc_k])
    
        return parallel_model
    else:
        model = Model(inputs=inp, outputs=x)
        model.compile(loss='binary_crossentropy', optimizer='adam', metrics=[mcc_k])
    
        return model

In [13]:
feature_config = {
    'time_steps': 300,
    'spectrogram': {
        'freq_bins': 50
    },
    'abs_rescale':1,
    'stats': {
        'mean': 1,
        'std': 1,
        'std_top': 1,
        'std_bot': 1,
        'max_range': 1,
        'percentiles': [0, 10, 25, 50, 75, 90, 100],
        'relative_percentiles': [0, 10, 25, 50, 75, 90, 100]
    }
}

model_config = {'hu1': 128, 'hu2': 64, 'dr1': 0.3, 'dr2':0.4, 'de1':64, 'parallel': False}

train_config = {'val_split': 1, 'stages_desc': [(500,2)]}

In [14]:
def train_model_base(X, y, tcf, model=model_one, model_config = model_config):
    
    val_split = tcf['val_split']
    stages_desc = tcf['stages_desc']
    
    
    t0 = time.time()
    stages = dict.fromkeys([i for i in range(len(stages_desc))])
    
    K.clear_session()
    model = model_one(X.shape, model_config)
    model.summary()

    for i in range(len(stages_desc)):
        stages[i] = {'batch_size':stages_desc[i][0], 'epochs': stages_desc[i][1]}
        hist_temp = model.fit(X, y, batch_size = stages_desc[i][0], 
                              epochs = stages_desc[i][1], 
                              validation_split = 0.2)
        stages[i]['results'] = hist_temp.history


    model_results = {'config': model.to_json(),
                     'training': stages
                    }

    best_score = max(model_results['training'][0]['results']['val_mcc_k'])
    print('='*100+'\n'+'='*100)
    elapsed_time = round(time.time()-t0, 3)/60.0
    print('Time elapsed: {} minutes... Best score: {} '.format(elapsed_time, best_score))
    
    return model_results, elapsed_time

In [15]:
        X = np.load('datasets/X_train.npy')
        y = np.load('datasets/y_train.npy')

In [16]:
out_, _ = train_model_base(X, y, train_config, model=model_one, model_config = model_config)

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 299, 207)          0         
_________________________________________________________________
bidirectional_1 (Bidirection (None, 299, 256)          344064    
_________________________________________________________________
dropout_1 (Dropout)          (None, 299, 256)          0         
_________________________________________________________________
bidirectional_2 (Bidirection (None, 299, 128)          164352    
_________________________________________________________________
attention_1 (Attention)      (None, 128)               427       
_________________________________________________________________
dense_1 (Dense)              (None, 64)                8256      
_________________________________________________________________
dropout_2 (Dropout)          (None, 64)                0         
__________

In [72]:
def hptuner(feature_config, model_config, train_config, 
            data_hyperparam_list=None, 
            model_hyperparam_list=None):
    
    check_hardware()
    meta_train, df_train = load_data()
    
    if (not data_hyperparam_list) and (not model_hyperparam_list):
        print('No parameters are being varied.')
        return
    if not data_hyperparam_list:
        data_hyperparam_list = [{'1': 1}]  
    if not model_hyperparam_list:
        model_hyperparam_list = [{'1': 1}]
        

    for data_hyperparameter_set in data_hyperparam_list:
        feature_config.update(data_hyperparameter_set)
        feature_pipe = FeatureExtractor(param_config=feature_config)
#         X,y = data_prep(meta_train, df_train, feature_pipe)
        X = np.load('datasets/X_train.npy')
        y = np.load('datasets/y_train.npy')
        print(X.shape, y.shape)
        
        for hyperparameter_set in model_hyperparam_list:
            
            model_config.update(hyperparameter_set)
            train_config.update(hyperparameter_set)
            
            t0 = time.time()
            out_, elapsed_time = train_model_base(X, y, train_config, 
                                                  model=model_one, 
                                                  model_config = model_config)
            out_['elapsed_time'] = elapsed_time
            out_['model_config'] = model_config
            out_['train_config'] = train_config
            out_['feature_config'] = feature_config
            
            with open(gen_name_v2('hpt')+'.pkl','wb') as fp:
                pickle.dump(out_, fp)

In [81]:
param_grid = {'stages_desc': [[(500,2)], [(600,4)]], 'val_split': [1,0.5]}
param_grid = list(ParameterGrid(param_grid))

In [86]:
data_grid = {'stages_desc': [[(500,2)], [(600,4)]], 'val_split': [1,0.5]}
data_grid = list(ParameterGrid(data_grid))
data_grid = None

In [74]:
hptuner(feature_config, model_config, train_config, model_hyperparam_list=param_grid, data_hyperparam_list=None)

Found 2 GPUs.
Found 16 CPU cores.
(2904, 299, 207) (2904,)
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 299, 207)          0         
_________________________________________________________________
bidirectional_1 (Bidirection (None, 299, 256)          344064    
_________________________________________________________________
dropout_1 (Dropout)          (None, 299, 256)          0         
_________________________________________________________________
bidirectional_2 (Bidirection (None, 299, 128)          164352    
_________________________________________________________________
attention_1 (Attention)      (None, 128)               427       
_________________________________________________________________
dense_1 (Dense)              (None, 64)                8256      
_________________________________________________________________
dropout_2 (Dropou

In [75]:
with open('models_v2/21905AA_hpt.pkl','rb') as fp:
    test = pickle.load(fp)